In [2]:
import warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim

from tw_dataset.dbmodels import *
import json

Switching to API Credentials #1


In [3]:
TM_MODELS_PATH = '/media/pablo/data/Tesis/models/old/tm_feats/'
from os.path import join

In [12]:
s = open_session()
all_tweets_es = [t for t in s.query(Tweet).all() if t.lang == 'es']
all_tweets_text_es = [t.text for t in all_tweets_es]

In [13]:
all_tweets_es_ids = [t.id for t in all_tweets_es]

In [14]:
len(all_tweets_text_es)    

1636480

In [15]:
# corpus = sample(all_tweets_text_es, 5000)
corpus = all_tweets_text_es

In [16]:
def preprocess(doc):
    pre_doc = doc
        
    # remover URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
    
    # minúsculas
    pre_doc = pre_doc.lower()

    # volar acentos
    pre_doc = gensim.utils.deaccent(pre_doc)

    # remove bullshit
    pre_doc = re.sub(r"\@|\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # contraer vocales
    for v in 'aeiou':
        pre_doc = re.sub(r"[%s]+" % v, v, pre_doc)    
    
    # normalizar espacio en blanco
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    return pre_doc

In [17]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation

spanish_tokenizer = load('tokenizers/punkt/spanish.pickle')

# stopwords en español
spanish_stopwords = stopwords.words('spanish')

# spanish stemmer
stemmer = SnowballStemmer('spanish')

# punctuation to remove
non_words = list(punctuation)

# we add spanish punctuation
non_words.extend(['¿', '¡'])
non_words.extend(map(str, range(10)))

stemmer = SnowballStemmer('spanish')

def trystem(t):
    try:
        t = stemmer.stem(t)
    except Exception:
        pass
    return t

def tokenize(text, stem=True, remove_stopwords=False):
    text = text.lower()
    result = []
    
    for sentence in spanish_tokenizer.tokenize(text):
        # remover puntuación
        text = ''.join([c for c in sentence if c not in non_words])
        
        # tokenize
        tokens = word_tokenize(text)

        if remove_stopwords:
            tokens = [t for t in tokens if t not in spanish_stopwords]

        # tokens de al menos 2 letras
        tokens = [t for t in tokens if len(t) > 1]
            
        # stem
        if stem:
            tokens = [trystem(t) for t in tokens]
        
        result += tokens
        
    return result

In [18]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [19]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc), remove_stopwords=True)
            yield tokens

In [20]:
# from gensim.models.phrases import Phraser, Phrases

In [21]:
# phrases = Phrases(get_docs(corpus), min_count=5)
# bigram = Phraser(phrases)
# trigram = Phrases(bigram[get_docs(corpus)], min_count=5)
# dictionary = gensim.corpora.Dictionary(trigram[get_docs(corpus)])

In [13]:
dictionary = gensim.corpora.Dictionary(get_docs(corpus))
dictionary.filter_extremes(no_below=100, no_above=0.3, keep_n=None)

In [22]:
sorted(dictionary.token2id.items(), key=lambda x:-x[1])[:50]

[(u'zocal', 11237),
 (u'podr', 11236),
 (u'coleccion', 11235),
 (u'chiquitit', 11234),
 (u'heidi', 11233),
 (u'experient', 11232),
 (u'antigued', 11231),
 (u'upahs', 11230),
 (u'santiag', 11229),
 (u'paritari', 11228),
 (u'gan', 11227),
 (u'paaristizabal', 11226),
 (u'torpez', 11225),
 (u'junt', 11224),
 (u'rellen', 11223),
 (u'kindl', 11222),
 (u'venci', 11221),
 (u'caciqu', 11220),
 (u'lanch', 11219),
 (u'peluqueri', 11218),
 (u'chavez', 11217),
 (u'juni', 11216),
 (u'palmir', 11215),
 (u'daliagutmann', 11214),
 (u'rial', 11213),
 (u'escriban', 11212),
 (u'mayweath', 11211),
 (u'infect', 11210),
 (u'volviendoacas', 11209),
 (u'pashkus', 11208),
 (u'vascarisnavarr', 11207),
 (u'chocolat', 11206),
 (u'aplic', 11205),
 (u'diplomat', 11204),
 (u'leandr', 11203),
 (u'cuelg', 11202),
 (u'kingmob', 11201),
 (u'cuell', 11200),
 (u'necesitari', 11199),
 (u'convencion', 11198),
 (u'fabiomb', 11197),
 (u'desvel', 11196),
 (u'consecuent', 11195),
 (u'bdlvictorhugoc', 11194),
 (u'ucr', 11193),
 (

In [4]:
dictionary = gensim.corpora.Dictionary.load(join(TM_MODELS_PATH,"tweets_es.dict"))

In [7]:
dictionary.filter_extremes(no_below=100)

In [8]:
len(dictionary.dfs)

11238

In [9]:
dictionary.save(join(TM_MODELS_PATH,"tweets_es.dict"))

In [23]:
bow = [dictionary.doc2bow(doc) for doc in get_docs(corpus)]

In [25]:
with open(join(TM_MODELS_PATH,'tweets_es_bow.pickle'), 'wb') as f:
    pickle.dump(bow,f)